# description-based-recommenderのサンプル

## 概要

映画の概要を利用し、その類似度によってデータを抽出するdescription-based-recommenderのサンプルコードを
以下に実装する

## 参考
Hands-on Recommendation Systems with Python

## データ取得

movies_metadata.csvからデータを取得する。
以下のデータを使用

id: 映画のID 
title: 映画タイトル
overview: 概要

In [1]:
import pandas as pd
orig_df= pd.read_csv('./csv/movies_metadata.csv', low_memory=False)

# MAXの数を出すとメモリが足らなくて落ちるので10000にしている
orig_df = orig_df[0:10000]
orig_df.shape

(10000, 24)

id、title、overviewを取り出す

In [2]:
df = pd.DataFrame({ 'id': orig_df['id'], 'title': orig_df['title'], 'overview': orig_df['overview']})
df.head()

,id,title,overview
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,8844,Jumanji,When siblings Judy and Peter discover an encha...
2,15602,Grumpier Old Men,A family wedding reignites the ancient feud be...
3,31357,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom..."
4,11862,Father of the Bride Part II,Just when George Banks has recovered from his ...


overivewからtef-idf matrixを作成する
各、単語に重み付けをして重要な単語同士で結び付けられるようにする

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfids = TfidfVectorizer(stop_words="english")

df['overview'] = df['overview'].fillna('')

tfidf_matrix = tfids.fit_transform(df['overview'])

tfidf_matrix.shape

(10000, 32350)

linear_kernelは類似度算出に使用

In [4]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [5]:
indicies = pd.Series(df.index, index=df['title']).drop_duplicates()

概要を見て、その類似度でソートを実行しTOP10を出力する

In [6]:
def recommend_movie(title, cosine_sim=cosine_sim, df=df, indicies=indicies):
    idx = indicies[title]
    
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1],  reverse=True)
    sim_scores = sim_scores[1:11]
    
    movie_indices = [i[0] for i in  sim_scores]
    
    return df['title'].iloc[movie_indices]

## 最終出力
The Wizard of Ozで検索してReturn to Ozの抽出に成功。

In [7]:
recommend_movie('The Wizard of Oz')

3753               Supergirl
4113           Just Visiting
5828     A Kiss Before Dying
1984            Return to Oz
359            The Lion King
8967                 Dollman
8061           Thunderbird 6
9698    Howl's Moving Castle
9214             Crane World
6893             The Witches
Name: title, dtype: object